In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [18]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [19]:
key_path = '../service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [20]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [21]:
# Create dataset reference
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')
# Retrieve dataset from reference
dataset = client.get_dataset(dataset_ref)

In [22]:
# Try if this connected

query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
LIMIT 5
"""

df = client.query(query).to_dataframe()
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5 non-null      Int64 
 2   visitId               5 non-null      Int64 
 3   visitStartTime        5 non-null      Int64 
 4   date                  5 non-null      object
 5   totals                5 non-null      object
 6   trafficSource         5 non-null      object
 7   device                5 non-null      object
 8   geoNetwork            5 non-null      object
 9   customDimensions      5 non-null      object
 10  hits                  5 non-null      object
 11  fullVisitorId         5 non-null      object
 12  userId                0 non-null      object
 13  channelGrouping       5 non-null      object
 14  socialEngagementType  5 non-null      object
dtypes: Int64(4), object(11)
memory usage: 752.0+

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


In [23]:
query = """
SELECT
  products.v2ProductCategory as category,
  products.v2ProductName as product,
  totals.visits as visits,
  totals.timeOnSite as time_on_site,
  totals.transactions as transactions,
  totals.transactionRevenue as revenue,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) products
WHERE 
  (_TABLE_SUFFIX BETWEEN '20170501' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
  (geoNetwork.country = 'United States')
  AND
  (
  products.v2ProductCategory LIKE '%Accessories%'
  OR products.v2ProductCategory LIKE '%Electronics%'
  OR products.v2ProductCategory LIKE '%Office%'
  OR products.v2ProductCategory LIKE "%Men's%"
  OR products.v2ProductCategory LIKE "%Women's%"
  OR products.v2ProductCategory LIKE "%Kid's%"
  OR products.v2ProductCategory LIKE "%Apparel%"
  OR products.v2ProductCategory LIKE '%Shop by Brand%'
  )
"""

df_totals = client.query(query).to_dataframe()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127178 entries, 0 to 127177
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   category      127178 non-null  object
 1   product       127178 non-null  object
 2   visits        127178 non-null  Int64 
 3   time_on_site  127073 non-null  Int64 
 4   transactions  57612 non-null   Int64 
 5   revenue       57612 non-null   Int64 
dtypes: Int64(4), object(2)
memory usage: 6.3+ MB
None


,category,product,visits,time_on_site,transactions,revenue
0,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,49,<NA>,<NA>
1,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,49,<NA>,<NA>
2,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,66,<NA>,<NA>
3,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,66,<NA>,<NA>
4,Home/Apparel/Kid's/Kid's-Infant/,Google Onesie Red/Graphite,1,39,<NA>,<NA>


In [24]:
df_totals = df_totals.dropna(axis=1, how='all').fillna(0)

print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127178 entries, 0 to 127177
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   category      127178 non-null  object
 1   product       127178 non-null  object
 2   visits        127178 non-null  Int64 
 3   time_on_site  127178 non-null  Int64 
 4   transactions  127178 non-null  Int64 
 5   revenue       127178 non-null  Int64 
dtypes: Int64(4), object(2)
memory usage: 6.3+ MB
None


,category,product,visits,time_on_site,transactions,revenue
0,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,49,0,0
1,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,49,0,0
2,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,66,0,0
3,Home/Shop by Brand/YouTube/,YouTube Leatherette Notebook Combo,1,66,0,0
4,Home/Apparel/Kid's/Kid's-Infant/,Google Onesie Red/Graphite,1,39,0,0


In [25]:
def categorized(x):
    
    def grouping_apparel(x):
        if x in ['Men\'s', 'Women\'s', 'Kid\'s', 'Apparel']:
            return 'Apparel'
        return x
    
    category = []
    categories = ['Accessories', 'Electronics', 'Office', 'Men\'s', 'Women\'s', 'Kid\'s', 'Apparel', 'Shop by Brand']
    for i in x:
        for cat in categories:
            if i.find(cat) != -1:
                category.append(cat)
    if len(category) == 0:
        return None
    
    result = grouping_apparel(category[-1])
    return result

df_totals['category'] = df_totals['category'].str.split('/').apply(categorized)
df_totals = df_totals.groupby(['category', 'product']).sum().reset_index()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      381 non-null    object
 1   product       381 non-null    object
 2   visits        381 non-null    Int64 
 3   time_on_site  381 non-null    Int64 
 4   transactions  381 non-null    Int64 
 5   revenue       381 non-null    Int64 
dtypes: Int64(4), object(2)
memory usage: 19.5+ KB
None


,category,product,visits,time_on_site,transactions,revenue
0,Accessories,1 oz Hand Sanitizer,225,270896,66,9887430000
1,Accessories,"7"" Dog Frisbee",102,109888,43,13088290000
2,Accessories,8 pc Android Sticker Sheet,366,373644,109,24754460000
3,Accessories,Android Luggage Tag,60,47670,0,0
4,Accessories,Android Lunch Kit,302,332494,32,3758170000


In [31]:
query = """
SELECT
  CASE 
    WHEN hits.eCommerceAction.action_type = '1' THEN 'Click through of product lists'
    WHEN hits.eCommerceAction.action_type = '2' THEN 'Product detail views'
    WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
    WHEN hits.eCommerceAction.action_type = '6' THEN 'Completed purchase'
  END as action,
  products.v2ProductCategory as category,
  products.v2ProductName as product,
  trafficSource.medium as medium,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) products
WHERE 
  (_TABLE_SUFFIX BETWEEN '20170501' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
    (geoNetwork.country = 'United States')
  AND
  (
  products.v2ProductCategory LIKE '%Accessories%'
  OR products.v2ProductCategory LIKE '%Electronics%'
  OR products.v2ProductCategory LIKE '%Office%'
  OR products.v2ProductCategory LIKE "%Men's%"
  OR products.v2ProductCategory LIKE "%Women's%"
  OR products.v2ProductCategory LIKE "%Kid's%"
  OR products.v2ProductCategory LIKE "%Apparel%"
  OR products.v2ProductCategory LIKE '%Shop by Brand%'
  )
"""

df_hits_traffic = client.query(query).to_dataframe()
print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127178 entries, 0 to 127177
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   action    127178 non-null  object
 1   category  127178 non-null  object
 2   product   127178 non-null  object
 3   medium    127178 non-null  object
dtypes: object(4)
memory usage: 3.9+ MB
None


,action,category,product,medium
0,Product detail views,Home/Electronics/,Micro Wireless Earbud,cpc
1,Click through of product lists,Home/Electronics/,Micro Wireless Earbud,cpc
2,Click through of product lists,Home/Electronics/,Micro Wireless Earbud,cpc
3,Product detail views,Home/Electronics/,Clip-on Compact Charger,cpc
4,Click through of product lists,Home/Electronics/,Clip-on Compact Charger,cpc


In [33]:
df_hits_traffic['category'] = df_hits_traffic['category'].str.split('/').apply(categorized)
df_ohe = pd.get_dummies(df_hits_traffic[['medium', 'action']], dtype=int)
df_hits_traffic = pd.concat([df_hits_traffic.drop(['medium', 'action'], axis=1),
                            df_ohe], axis=1)

def modify_columns(array):
    temp = []
    for d in array:
        temp.append(d.replace('(', '').replace(')', '').replace(' ', '_'))
    return temp


df_hits_traffic.columns = modify_columns(np.array(df_hits_traffic.columns))

df_hits_traffic = df_hits_traffic.groupby(['category', 'product']).sum().reset_index()

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   category                               381 non-null    object
 1   product                                381 non-null    object
 2   medium_none                            381 non-null    int32 
 3   medium_affiliate                       381 non-null    int32 
 4   medium_cpc                             381 non-null    int32 
 5   medium_cpm                             381 non-null    int32 
 6   medium_organic                         381 non-null    int32 
 7   medium_referral                        381 non-null    int32 
 8   action_Check_out                       381 non-null    int32 
 9   action_Click_through_of_product_lists  381 non-null    int32 
 10  action_Completed_purchase              381 non-null    int32 
 11  action_Product_deta

,category,product,medium_none,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Accessories,1 oz Hand Sanitizer,122,0,21,0,77,5,0,121,0,104
1,Accessories,"7"" Dog Frisbee",65,0,2,0,33,2,0,58,0,44
2,Accessories,8 pc Android Sticker Sheet,228,2,11,2,105,18,0,196,0,170
3,Accessories,Android Luggage Tag,31,0,4,2,19,4,0,32,0,28
4,Accessories,Android Lunch Kit,148,10,26,3,102,13,0,177,0,125


In [34]:
df = pd.merge(df_totals, df_hits_traffic, how='outer', on=['category', 'product'])
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381 entries, 0 to 380
Data columns (total 16 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   category                               381 non-null    object
 1   product                                381 non-null    object
 2   visits                                 381 non-null    Int64 
 3   time_on_site                           381 non-null    Int64 
 4   transactions                           381 non-null    Int64 
 5   revenue                                381 non-null    Int64 
 6   medium_none                            381 non-null    int32 
 7   medium_affiliate                       381 non-null    int32 
 8   medium_cpc                             381 non-null    int32 
 9   medium_cpm                             381 non-null    int32 
 10  medium_organic                         381 non-null    int32 
 11  medium_referral    

,category,product,visits,time_on_site,transactions,revenue,medium_none,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Accessories,1 oz Hand Sanitizer,225,270896,66,9887430000,122,0,21,0,77,5,0,121,0,104
1,Accessories,"7"" Dog Frisbee",102,109888,43,13088290000,65,0,2,0,33,2,0,58,0,44
2,Accessories,8 pc Android Sticker Sheet,366,373644,109,24754460000,228,2,11,2,105,18,0,196,0,170
3,Accessories,Android Luggage Tag,60,47670,0,0,31,0,4,2,19,4,0,32,0,28
4,Accessories,Android Lunch Kit,302,332494,32,3758170000,148,10,26,3,102,13,0,177,0,125


In [35]:
categories = df['category'].unique()
selected = []

for cat in categories:
    temp = df[df['category'] == cat]
    top10 = list(temp.sort_values(by='transactions', ascending=False).head(10)['product'].values)
    selected = selected + top10

df_result = df[df['product'].isin(selected)]

In [37]:
df_result.to_csv('../result/top_product.csv', index=False)

In [40]:
df_result[df_result['category'] == 'Shop by Brand'].head().sum()

category                                 Shop by BrandShop by BrandShop by BrandShop by...
product                                  22 oz YouTube Bottle Infuser24 oz YouTube Serg...
visits                                                                                 884
time_on_site                                                                        673820
transactions                                                                           109
revenue                                                                        18603530000
medium_none                                                                            303
medium_affiliate                                                                         4
medium_cpc                                                                              18
medium_cpm                                                                               0
medium_organic                                                                         503